In [43]:
!pip install opendatasets

In [44]:
import opendatasets as op
op.download('https://www.kaggle.com/datasets/marquis03/bean-leaf-lesions-classification')

Skipping, found downloaded files in "./bean-leaf-lesions-classification" (use force=True to force download)


In [45]:
import torch, torchvision, PIL
import os
import pandas

In [46]:
class_name = ['healthy','angular_leaf_spot','bean_rust']
class_id = [0,1,2]

In [47]:
df = pandas.read_csv('/content/bean-leaf-lesions-classification/train.csv')
df.head(5)

,image:FILE,category
0,train/healthy/healthy_train.98.jpg,0
1,train/healthy/healthy_train.148.jpg,0
2,train/healthy/healthy_train.306.jpg,0
3,train/healthy/healthy_train.305.jpg,0
4,train/healthy/healthy_train.40.jpg,0


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034 entries, 0 to 1033
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image:FILE  1034 non-null   object
 1   category    1034 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 16.3+ KB


In [49]:
df['category'].value_counts()

2    348
1    345
0    341
Name: category, dtype: int64

In [50]:
class Leaf_Dataset(torch.utils.data.Dataset):
  def __init__(self,root_path,df, transform = None):
    self.root_path = root_path
    self.df = df
    self.images = self._get_images()

  def _get_images(self):
    images = []
    for row in self.df.iloc:
      name = row['image:FILE']
      label = row['category']
      image_path = os.path.join(self.root_path,name)
      images.append((image_path, label))
    return images

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    img_path, label = self.images[idx]
    img = PIL.Image.open(img_path)
    if transform:
      img = transform(img)
    return img, label

In [51]:
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((200,200)),
                                            torchvision.transforms.ToTensor()])

In [52]:
dataset = Leaf_Dataset('/content/bean-leaf-lesions-classification', df, transform)

In [53]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle = True)

In [54]:
img, label = dataset[0]

In [55]:
img

tensor([[[0.0510, 0.0627, 0.0627,  ..., 0.1490, 0.1843, 0.2314],
         [0.0863, 0.0863, 0.0824,  ..., 0.1255, 0.1686, 0.2118],
         [0.1373, 0.1333, 0.1255,  ..., 0.1294, 0.1804, 0.2078],
         ...,
         [0.6941, 0.6863, 0.6588,  ..., 0.1333, 0.1451, 0.2157],
         [0.6980, 0.7137, 0.6667,  ..., 0.1333, 0.1412, 0.1765],
         [0.6980, 0.6941, 0.6588,  ..., 0.1294, 0.1490, 0.1725]],

        [[0.2039, 0.1961, 0.1961,  ..., 0.3804, 0.4196, 0.4706],
         [0.2627, 0.2471, 0.2235,  ..., 0.3725, 0.4196, 0.4627],
         [0.3490, 0.3216, 0.2784,  ..., 0.3882, 0.4392, 0.4627],
         ...,
         [0.5373, 0.5255, 0.4863,  ..., 0.2431, 0.2667, 0.3451],
         [0.5412, 0.5490, 0.4941,  ..., 0.2392, 0.2588, 0.3059],
         [0.5412, 0.5294, 0.4902,  ..., 0.2392, 0.2667, 0.3020]],

        [[0.0039, 0.0000, 0.0000,  ..., 0.0431, 0.0667, 0.1294],
         [0.0196, 0.0118, 0.0000,  ..., 0.0196, 0.0588, 0.1216],
         [0.0667, 0.0510, 0.0275,  ..., 0.0078, 0.0627, 0.

In [56]:
label

0

In [57]:
class Leaf_Health_Model(torchvision.models.DenseNet):
  def __init__(self):
    super(Leaf_Health_Model,self).__init__()
    pretrained_weigth = torchvision.models.densenet121(pretrained = True).state_dict()
    self.load_state_dict(pretrained_weigth)
    self.classifier = torch.nn.Linear(1024,3, bias = True)

  def forward(self,X):
    o = super(Leaf_Health_Model,self).forward(X)
    return o

In [58]:
model = Leaf_Health_Model()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [59]:
state_dict = torch.load('/content/drive/MyDrive/Project/Leaf_Health.pth')

In [60]:
model.load_state_dict(state_dict['model'])

<All keys matched successfully>

In [61]:
model

Leaf_Health_Model(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [62]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [ ]:
for epoch in range(5):
  c = 0
  for img, label in dataloader:
    o = model(img)

    target = torch.tensor(label)
    loss = criterion(o, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if c%10 == 0:
      print(o.argmax(dim = 1), label)
      print(f'epoch: {epoch}, iter: {c}, loss : {loss.item()}')
    c += 1

<ipython-input-254-560a43464626>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(label)


tensor([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([0, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 2, 0, 0, 1, 0])
epoch: 0, iter: 0, loss : 1.1326647996902466
tensor([2, 2, 0, 2, 1, 2, 0, 1, 1, 0, 2, 2, 2, 0, 0, 1]) tensor([2, 2, 0, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1])
epoch: 0, iter: 10, loss : 0.8756710290908813
tensor([0, 2, 1, 2, 0, 2, 0, 1, 1, 2, 1, 0, 2, 1, 0, 0]) tensor([0, 2, 2, 1, 0, 2, 0, 1, 1, 2, 1, 2, 0, 1, 0, 0])
epoch: 0, iter: 20, loss : 0.49826768040657043
tensor([1, 2, 1, 0, 2, 1, 2, 2, 2, 2, 0, 1, 2, 0, 0, 1]) tensor([1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 0, 0, 1])
epoch: 0, iter: 30, loss : 0.8507419228553772
tensor([1, 2, 2, 0, 1, 2, 0, 0, 1, 1, 0, 2, 1, 2, 1, 2]) tensor([1, 2, 2, 0, 2, 0, 0, 0, 1, 1, 0, 0, 2, 2, 1, 2])
epoch: 0, iter: 40, loss : 0.3763388991355896
tensor([2, 1, 0, 0, 0, 1, 0, 1, 1, 0, 2, 2, 0, 0, 2, 0]) tensor([2, 1, 0, 0, 0, 1, 0, 1, 1, 0, 2, 2, 0, 0, 2, 0])
epoch: 0, iter: 50, loss : 0.07755105197429657
tensor([0, 0, 0, 1, 2, 1, 2, 2, 0, 1, 1

In [63]:
val_df = pandas.read_csv('/content/bean-leaf-lesions-classification/val.csv')

In [70]:
val_dataset = Leaf_Dataset('/content/bean-leaf-lesions-classification', val_df, transform)

In [72]:
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 16, shuffle = True)

In [73]:
img, label = dataset[10]

In [74]:
# torch.save({'model' : model.state_dict(),
#             'optimizer' : optimizer.state_dict()}, '/content/drive/MyDrive/Project/Leaf_Health.pth')

In [75]:
result = []
labels = []
for img, label in val_dataloader:
  o = model(img)

  target = torch.tensor(label)
  loss = criterion(o, target)
  result += o.argmax(dim = -1).tolist()
  labels += label.tolist()
  # print(o.argmax(dim = 1), label)
  # print(f'loss: {loss.item()}, acc: {len([x for x in o.argmax(dim = -1).tolist() if x in label.tolist()])/len(label)}')

<ipython-input-75-b566f0d7d418>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(label)


In [76]:
confusion_matrix = [[0 for _ in range(3)] for _ in range(3)]
confusion_matrix

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

In [77]:
for pred, target in zip(result,labels):
  confusion_matrix[pred][target] += 1


In [78]:
for row in confusion_matrix:
  print(row)

[42, 0, 1]
[1, 42, 1]
[1, 2, 43]
